In [40]:
## The only packages we would need for this pipeline are `Requests`, `os`, `Pandas`, `SQLite`, and `Streamlit`.
## Requests is to make HTTP requests to fetch the data from REST API endpoints
## And Pandas is for data transformations and wrangling.

import requests
import pandas as pd
import os

key = os.getenv('API_KEY')

response = requests.get("https://jsonplaceholder.typicode.com/users").json()

columns=['customer_id',
        'name',
        'username',
        'email',
        'lat',
        'lng']

users = pd.DataFrame(columns=columns)
i = 0

for rec in response:
    users.loc[i] = ([rec['id'],
                     rec['name'],
                     rec['username'], 
                     rec['email'],
                     rec['address']['geo']['lat'],
                     rec['address']['geo']['lng']])
    i += 1

sales = pd.read_csv('./data/sales_data.csv').merge(users, on='customer_id')